In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alici\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [6]:
from imblearn.over_sampling import SMOTE
from collections import Counter

In [7]:
from sklearn.model_selection import GridSearchCV

In [8]:
def createdfs(i):
    df_train = pd.read_csv("C:/Users/alici/Documents/tcc/github2/tcc-v2/b5/particoes/k_"+str(i)+"/train.csv")
    df_test = pd.read_csv("C:/Users/alici/Documents/tcc/github2/tcc-v2/b5/particoes/k_"+str(i)+"/test.csv")
    df_train["Transf_Text"] = df_train["Text"].apply((lambda x: (str(x)).split("#")))
    df_test["Transf_Text"] = df_test["Text"].apply((lambda x: (str(x)).split("#")))
    return df_train, df_test

In [9]:
def remove_stopwords(texts):
    result = []
    for text in texts:
        all_words = text.split(" ")
        clean_text = [i for i in all_words if i not in stopwords and i!=""]
        result.append(" ".join(clean_text))
    return result

In [10]:
def getstringtext(df):
    texts = df["CleanText"].to_list()
    string_text = []
    for l in texts:
        res = " ".join(l)
        res = res.replace("\"", "")
        string_text.append(res)
    return string_text

In [11]:
def getmetrics(y_test, y_pred):
    mc = metrics.confusion_matrix(y_test, y_pred)
    a = metrics.accuracy_score(y_test, y_pred)
    p = metrics.precision_score(y_test, y_pred, average=None)
    r = metrics.recall_score(y_test, y_pred, average=None)
    f1 = metrics.f1_score(y_test, y_pred, average=None)
    return mc, a, p, r, f1

In [12]:
def genderGroups(text):
    if(text == "f"):
        return 1
    else:
        return 0

In [13]:
def mean(ages_list):
    soma = [0, 0, 0]
    for a1, a2, a3 in ages_list:
        soma[0] += a1
        soma[1] += a2
        soma[2] += a3
    return [(soma[0]/len(ages_list)), (soma[1]/len(ages_list)), (soma[2]/len(ages_list))]

In [14]:
def formatTime(seg):
    min = seg//60
    seg = seg % 60
    return str(min)+"min"+str(round(seg, 2))+"s"

In [15]:
def mean_macro(values):
    return sum(values)/len(values)

In [16]:
import json

In [28]:
with open('b5/gender.json', encoding='utf-8') as f:
    gender_param = json.load(f)

In [33]:
with open('b5/age.json', encoding='utf-8') as f:
    age_param = json.load(f)

In [36]:
with open('b5/age_fem.json', encoding='utf-8') as f:
    age_fem_param = json.load(f)

In [50]:
with open('b5/age_masc.json', encoding='utf-8') as f:
    age_masc_param = json.load(f)

In [21]:
age_masc_param["k_1"]["params"]["penalty"]

'l2'

In [53]:
ages = []
ages_fem = []
ages_masc = []
ages_sep = []

ages_macro = []
ages_fem_macro = []
ages_masc_macro = []
ages_sep_macro = []

for i in range(10):
    df_train_gender, df_test_gender = createdfs(i+1)

    df_train_gender['CleanText'] = df_train_gender['Transf_Text'].apply(remove_stopwords)
    df_test_gender['CleanText'] = df_test_gender['Transf_Text'].apply(remove_stopwords)

    df_train_gender["GenderClass"] = df_train_gender["ActualGender"].apply(genderGroups)
    df_test_gender["GenderClass"] = df_test_gender["ActualGender"].apply(genderGroups)

    train_texts_gender = getstringtext(df_train_gender)
    test_texts_gender = getstringtext(df_test_gender)

    tfvec = TfidfVectorizer(max_features = 3000)
    tfvec.fit(train_texts_gender)
    tdf_train = tfvec.transform(train_texts_gender).toarray().tolist()
    tdf_test = tfvec.transform(test_texts_gender).toarray().tolist()

    gender_train = df_train_gender["GenderClass"].to_list()
    gender_test = df_test_gender["GenderClass"].to_list()

    logisticRegr_gender = LogisticRegression(penalty=gender_param["k_"+str(i+1)]["params"]["penalty"], C=gender_param["k_"+str(i+1)]["params"]["C"])
    logisticRegr_gender.fit(tdf_train, gender_train)

    pred_gender_test=logisticRegr_gender.predict(tdf_test)

    #genero
    df_test_gender["PredictGender"] = pred_gender_test

    # separa as partições de teste com base no predito

    df_train_fem = df_train_gender[df_train_gender["GenderClass"]==1]
    df_train_masc = df_train_gender[df_train_gender["GenderClass"]==0]

    df_test_fem = df_test_gender[df_test_gender["PredictGender"]==1]
    df_test_masc = df_test_gender[df_test_gender["PredictGender"]==0]

    age_train = df_train_gender["AgeClass"].to_list()
    age_test = df_test_gender["AgeClass"].to_list()
    logisticRegr = LogisticRegression(penalty=age_param["k_"+str(i+1)]["params"]["penalty"], C=age_param["k_"+str(i+1)]["params"]["C"])
    logisticRegr.fit(tdf_train, age_train)
    pred_age=logisticRegr.predict(tdf_test)
    ages_macro.append(metrics.f1_score(age_test, pred_age, average='macro'))
    mc, a, p, r, f1 = getmetrics(age_test, pred_age)
    ages.append(f1)

    train_texts_fem = getstringtext(df_train_fem)
    test_texts_fem = getstringtext(df_test_fem)
    tfvec_fem = TfidfVectorizer(max_features = 3000)
    tfvec_fem.fit(train_texts_fem)
    tdf_train_fem = tfvec_fem.transform(train_texts_fem).toarray().tolist()
    tdf_test_fem = tfvec_fem.transform(test_texts_fem).toarray().tolist()
    age_train_fem = df_train_fem["AgeClass"].to_list()
    age_test_fem = df_test_fem["AgeClass"].to_list()
    logisticRegr_fem = LogisticRegression(penalty=age_fem_param["k_"+str(i+1)]["params"]["penalty"], C=age_fem_param["k_"+str(i+1)]["params"]["C"])
    logisticRegr_fem.fit(tdf_train_fem, age_train_fem)
    pred_age_fem=logisticRegr_fem.predict(tdf_test_fem)
    ages_fem_macro.append(metrics.f1_score(age_test_fem, pred_age_fem, average='macro'))
    mc, a, p, r, f1_fem = getmetrics(age_test_fem, pred_age_fem)
    ages_fem.append(f1_fem)

    train_texts_masc = getstringtext(df_train_masc)
    test_texts_masc = getstringtext(df_test_masc)
    tfvec_masc = TfidfVectorizer(max_features = 3000)
    tfvec_masc.fit(train_texts_masc)
    tdf_train_masc = tfvec_masc.transform(train_texts_masc).toarray().tolist()
    tdf_test_masc = tfvec_masc.transform(test_texts_masc).toarray().tolist()
    age_train_masc = df_train_masc["AgeClass"].to_list()
    age_test_masc = df_test_masc["AgeClass"].to_list()
    logisticRegr_masc = LogisticRegression(penalty=age_masc_param["k_"+str(i+1)]["params"]["penalty"], C=age_masc_param["k_"+str(i+1)]["params"]["C"])
    logisticRegr_masc.fit(tdf_train_masc, age_train_masc)
    pred_age_masc=logisticRegr_masc.predict(tdf_test_masc)
    ages_masc_macro.append(metrics.f1_score(age_test_masc, pred_age_masc, average='macro'))
    mc, a, p, r, f1_masc = getmetrics(age_test_masc, pred_age_masc)
    ages_masc.append(f1_masc)

    pred_ages_sep = pred_age_fem.tolist() + pred_age_masc.tolist()
    y_test = age_test_fem + age_test_masc
    mc, a, p, r, f1_sep = getmetrics(y_test, pred_ages_sep)
    ages_sep.append(f1_sep)
    ages_sep_macro.append(metrics.f1_score(y_test, pred_ages_sep, average='macro'))

print()
print("juntos:", end=" ")
print(sum(ages_macro)/len(ages_macro))
print("só fem:", end=" ")
print(sum(ages_fem_macro)/len(ages_fem_macro))
print("só masc:", end=" ")
print(sum(ages_masc_macro)/len((ages_masc_macro)))
print("separado:", end=" ")
print(sum(ages_sep_macro)/len(ages_sep_macro))

c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (m


juntos: 0.5709021454700153
só fem: 0.5812601959625749
só masc: 0.5143461941109
separado: 0.5748136713335728


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [54]:
ages = []
ages_fem = []
ages_masc = []
ages_sep = []

ages_macro = []
ages_fem_macro = []
ages_masc_macro = []
ages_sep_macro = []

for i in range(10):
    df_train_gender, df_test_gender = createdfs(i+1)

    df_train_gender['CleanText'] = df_train_gender['Transf_Text'].apply(remove_stopwords)
    df_test_gender['CleanText'] = df_test_gender['Transf_Text'].apply(remove_stopwords)

    df_train_gender["GenderClass"] = df_train_gender["ActualGender"].apply(genderGroups)
    df_test_gender["GenderClass"] = df_test_gender["ActualGender"].apply(genderGroups)

    train_texts_gender = getstringtext(df_train_gender)
    test_texts_gender = getstringtext(df_test_gender)

    tfvec = TfidfVectorizer(max_features = 3000)
    tfvec.fit(train_texts_gender)
    tdf_train = tfvec.transform(train_texts_gender).toarray().tolist()
    tdf_test = tfvec.transform(test_texts_gender).toarray().tolist()

    gender_train = df_train_gender["GenderClass"].to_list()
    gender_test = df_test_gender["GenderClass"].to_list()

    logisticRegr_gender = LogisticRegression(penalty="l2", C=1000)
    logisticRegr_gender.fit(tdf_train, gender_train)

    pred_gender_test=logisticRegr_gender.predict(tdf_test)

    #genero
    df_test_gender["PredictGender"] = pred_gender_test

    # separa as partições de teste com base no predito

    df_train_fem = df_train_gender[df_train_gender["GenderClass"]==1]
    df_train_masc = df_train_gender[df_train_gender["GenderClass"]==0]

    df_test_fem = df_test_gender[df_test_gender["PredictGender"]==1]
    df_test_masc = df_test_gender[df_test_gender["PredictGender"]==0]

    age_train = df_train_gender["AgeClass"].to_list()
    age_test = df_test_gender["AgeClass"].to_list()
    logisticRegr = LogisticRegression(penalty="l2", C=1000)
    logisticRegr.fit(tdf_train, age_train)
    pred_age=logisticRegr.predict(tdf_test)
    ages_macro.append(metrics.f1_score(age_test, pred_age, average='macro'))
    mc, a, p, r, f1 = getmetrics(age_test, pred_age)
    ages.append(f1)

    train_texts_fem = getstringtext(df_train_fem)
    test_texts_fem = getstringtext(df_test_fem)
    tfvec_fem = TfidfVectorizer(max_features = 3000)
    tfvec_fem.fit(train_texts_fem)
    tdf_train_fem = tfvec_fem.transform(train_texts_fem).toarray().tolist()
    tdf_test_fem = tfvec_fem.transform(test_texts_fem).toarray().tolist()
    age_train_fem = df_train_fem["AgeClass"].to_list()
    age_test_fem = df_test_fem["AgeClass"].to_list()
    logisticRegr_fem = LogisticRegression(penalty="l2", C=1000)
    logisticRegr_fem.fit(tdf_train_fem, age_train_fem)
    pred_age_fem=logisticRegr_fem.predict(tdf_test_fem)
    ages_fem_macro.append(metrics.f1_score(age_test_fem, pred_age_fem, average='macro'))
    mc, a, p, r, f1_fem = getmetrics(age_test_fem, pred_age_fem)
    ages_fem.append(f1_fem)

    train_texts_masc = getstringtext(df_train_masc)
    test_texts_masc = getstringtext(df_test_masc)
    tfvec_masc = TfidfVectorizer(max_features = 3000)
    tfvec_masc.fit(train_texts_masc)
    tdf_train_masc = tfvec_masc.transform(train_texts_masc).toarray().tolist()
    tdf_test_masc = tfvec_masc.transform(test_texts_masc).toarray().tolist()
    age_train_masc = df_train_masc["AgeClass"].to_list()
    age_test_masc = df_test_masc["AgeClass"].to_list()
    logisticRegr_masc = LogisticRegression(penalty="l2", C=1000)
    logisticRegr_masc.fit(tdf_train_masc, age_train_masc)
    pred_age_masc=logisticRegr_masc.predict(tdf_test_masc)
    ages_masc_macro.append(metrics.f1_score(age_test_masc, pred_age_masc, average='macro'))
    mc, a, p, r, f1_masc = getmetrics(age_test_masc, pred_age_masc)
    ages_masc.append(f1_masc)

    pred_ages_sep = pred_age_fem.tolist() + pred_age_masc.tolist()
    y_test = age_test_fem + age_test_masc
    mc, a, p, r, f1_sep = getmetrics(y_test, pred_ages_sep)
    ages_sep.append(f1_sep)
    ages_sep_macro.append(metrics.f1_score(y_test, pred_ages_sep, average='macro'))

print()
print("juntos:", end=" ")
print(sum(ages_macro)/len(ages_macro))
print("só fem:", end=" ")
print(sum(ages_fem_macro)/len(ages_fem_macro))
print("só masc:", end=" ")
print(sum(ages_masc_macro)/len((ages_masc_macro)))
print("separado:", end=" ")
print(sum(ages_sep_macro)/len(ages_sep_macro))

c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab


juntos: 0.570935598217772
só fem: 0.5801392419899586
só masc: 0.4940126387773446
separado: 0.5712139794233195


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [29]:
gender = []

for i in range(10):
    df_train_gender, df_test_gender = createdfs(i+1)

    df_train_gender['CleanText'] = df_train_gender['Transf_Text'].apply(remove_stopwords)
    df_test_gender['CleanText'] = df_test_gender['Transf_Text'].apply(remove_stopwords)

    df_train_gender["GenderClass"] = df_train_gender["ActualGender"].apply(genderGroups)
    df_test_gender["GenderClass"] = df_test_gender["ActualGender"].apply(genderGroups)

    train_texts_gender = getstringtext(df_train_gender)
    test_texts_gender = getstringtext(df_test_gender)

    tfvec = TfidfVectorizer(max_features = 3000)
    tfvec.fit(train_texts_gender)
    tdf_train = tfvec.transform(train_texts_gender).toarray().tolist()
    tdf_test = tfvec.transform(test_texts_gender).toarray().tolist()

    gender_train = df_train_gender["GenderClass"].to_list()
    gender_test = df_test_gender["GenderClass"].to_list()

    logisticRegr_gender = LogisticRegression(penalty=gender_param["k_"+str(i+1)]["params"]["penalty"], C=gender_param["k_"+str(i+1)]["params"]["C"])
    logisticRegr_gender.fit(tdf_train, gender_train)

    pred_gender_test=logisticRegr_gender.predict(tdf_test)
    gender.append(metrics.f1_score(gender_test, pred_gender_test, average='macro'))
    print(metrics.f1_score(gender_test, pred_gender_test, average='macro'))

print(sum(gender)/len(gender))

0.9020715630885123
0.8153846153846154
0.900876858558902


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9226190476190477
0.8440779610194902
0.8263450834879407
0.8641284061216872
0.9013539651837523
0.8812111801242236
0.8819444444444444
0.8740013125032616


In [24]:
gender = []

for i in range(10):
    df_train_gender, df_test_gender = createdfs(i+1)

    df_train_gender['CleanText'] = df_train_gender['Transf_Text'].apply(remove_stopwords)
    df_test_gender['CleanText'] = df_test_gender['Transf_Text'].apply(remove_stopwords)

    df_train_gender["GenderClass"] = df_train_gender["ActualGender"].apply(genderGroups)
    df_test_gender["GenderClass"] = df_test_gender["ActualGender"].apply(genderGroups)

    train_texts_gender = getstringtext(df_train_gender)
    test_texts_gender = getstringtext(df_test_gender)

    tfvec = TfidfVectorizer(max_features = 3000)
    tfvec.fit(train_texts_gender)
    tdf_train = tfvec.transform(train_texts_gender).toarray().tolist()
    tdf_test = tfvec.transform(test_texts_gender).toarray().tolist()

    gender_train = df_train_gender["GenderClass"].to_list()
    gender_test = df_test_gender["GenderClass"].to_list()

    logisticRegr_gender = LogisticRegression(penalty='l2', C=1000)
    logisticRegr_gender.fit(tdf_train, gender_train)

    pred_gender_test=logisticRegr_gender.predict(tdf_test)
    gender.append(metrics.f1_score(gender_test, pred_gender_test, average='macro'))
    print(metrics.f1_score(gender_test, pred_gender_test, average='macro'))

print(sum(gender)/len(gender))

0.9020715630885123
0.8153846153846154


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.900876858558902


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9226190476190477
0.8440779610194902
0.845925925925926


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8641284061216872
0.9013539651837523


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8812111801242236
0.8819444444444444
0.8759593967470602


In [35]:
ages = []
ages_fem = []
ages_masc = []
ages_sep = []

ages_macro = []
ages_fem_macro = []
ages_masc_macro = []
ages_sep_macro = []

for i in range(10):
    df_train_gender, df_test_gender = createdfs(i+1)

    df_train_gender['CleanText'] = df_train_gender['Transf_Text'].apply(remove_stopwords)
    df_test_gender['CleanText'] = df_test_gender['Transf_Text'].apply(remove_stopwords)

    train_texts_gender = getstringtext(df_train_gender)
    test_texts_gender = getstringtext(df_test_gender)

    tfvec = TfidfVectorizer(max_features = 3000)
    tfvec.fit(train_texts_gender)
    tdf_train = tfvec.transform(train_texts_gender).toarray().tolist()
    tdf_test = tfvec.transform(test_texts_gender).toarray().tolist()

    age_train = df_train_gender["AgeClass"].to_list()
    age_test = df_test_gender["AgeClass"].to_list()
    logisticRegr = LogisticRegression(penalty=age_param["k_"+str(i+1)]["params"]["penalty"], C=age_param["k_"+str(i+1)]["params"]["C"])
    logisticRegr.fit(tdf_train, age_train)
    pred_age=logisticRegr.predict(tdf_test)
    ages_macro.append(metrics.f1_score(age_test, pred_age, average='macro'))
    print(metrics.f1_score(age_test, pred_age, average='macro'))
    
print(sum(ages_macro)/len(ages_macro))

0.5936507936507937


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6664497831164499


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.626081560283688


...

c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.48821548821548816


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5150375939849624
0.6104302832244008
0.5709021454700153


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [32]:
ages = []
ages_fem = []
ages_masc = []
ages_sep = []

ages_macro = []
ages_fem_macro = []
ages_masc_macro = []
ages_sep_macro = []

for i in range(10):
    df_train_gender, df_test_gender = createdfs(i+1)

    df_train_gender['CleanText'] = df_train_gender['Transf_Text'].apply(remove_stopwords)
    df_test_gender['CleanText'] = df_test_gender['Transf_Text'].apply(remove_stopwords)

    train_texts_gender = getstringtext(df_train_gender)
    test_texts_gender = getstringtext(df_test_gender)

    tfvec = TfidfVectorizer(max_features = 3000)
    tfvec.fit(train_texts_gender)
    tdf_train = tfvec.transform(train_texts_gender).toarray().tolist()
    tdf_test = tfvec.transform(test_texts_gender).toarray().tolist()

    age_train = df_train_gender["AgeClass"].to_list()
    age_test = df_test_gender["AgeClass"].to_list()
    logisticRegr = LogisticRegression(penalty="l2", C=100)
    logisticRegr.fit(tdf_train, age_train)
    pred_age=logisticRegr.predict(tdf_test)
    ages_macro.append(metrics.f1_score(age_test, pred_age, average='macro'))
    print(metrics.f1_score(age_test, pred_age, average='macro'))
    
print(sum(ages_macro)/len(ages_macro))

c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5875


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6490196078431373


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.48821548821548816


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5706563706563706
0.5714013279936201
0.5716501567454882


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [37]:
ages = []

for i in range(10):
    df_train_gender, df_test_gender = createdfs(i+1)

    df_train_gender['CleanText'] = df_train_gender['Transf_Text'].apply(remove_stopwords)
    df_test_gender['CleanText'] = df_test_gender['Transf_Text'].apply(remove_stopwords)

    df_train_gender["GenderClass"] = df_train_gender["ActualGender"].apply(genderGroups)
    df_test_gender["GenderClass"] = df_test_gender["ActualGender"].apply(genderGroups)

    train_texts_gender = getstringtext(df_train_gender)
    test_texts_gender = getstringtext(df_test_gender)

    tfvec = TfidfVectorizer(max_features = 3000)
    tfvec.fit(train_texts_gender)
    tdf_train = tfvec.transform(train_texts_gender).toarray().tolist()
    tdf_test = tfvec.transform(test_texts_gender).toarray().tolist()

    gender_train = df_train_gender["GenderClass"].to_list()
    gender_test = df_test_gender["GenderClass"].to_list()

    logisticRegr_gender = LogisticRegression(penalty=gender_param["k_"+str(i+1)]["params"]["penalty"], C=gender_param["k_"+str(i+1)]["params"]["C"])
    logisticRegr_gender.fit(tdf_train, gender_train)

    pred_gender_test=logisticRegr_gender.predict(tdf_test)

    #genero
    df_test_gender["PredictGender"] = pred_gender_test

    # separa as partições de teste com base no predito

    df_train_fem = df_train_gender[df_train_gender["GenderClass"]==1]
    df_test_fem = df_test_gender[df_test_gender["PredictGender"]==1]

    train_texts_fem = getstringtext(df_train_fem)
    test_texts_fem = getstringtext(df_test_fem)
    tfvec_fem = TfidfVectorizer(max_features = 3000)
    tfvec_fem.fit(train_texts_fem)
    tdf_train_fem = tfvec_fem.transform(train_texts_fem).toarray().tolist()
    tdf_test_fem = tfvec_fem.transform(test_texts_fem).toarray().tolist()
    age_train_fem = df_train_fem["AgeClass"].to_list()
    age_test_fem = df_test_fem["AgeClass"].to_list()
    logisticRegr_fem = LogisticRegression(penalty=age_fem_param["k_"+str(i+1)]["params"]["penalty"], C=age_fem_param["k_"+str(i+1)]["params"]["C"])
    logisticRegr_fem.fit(tdf_train_fem, age_train_fem)
    pred_age_fem=logisticRegr_fem.predict(tdf_test_fem)
    ages.append(metrics.f1_score(age_test_fem, pred_age_fem, average='macro'))
    print(metrics.f1_score(age_test_fem, pred_age_fem, average='macro'))

print(sum(ages)/len(ages))

c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


0.7587052062204193


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7036785512167515


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


...

c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5208647561588737


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


0.563973063973064
0.5969634230503796
0.5812601959625749


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [39]:
ages = []

for i in range(10):
    df_train_gender, df_test_gender = createdfs(i+1)

    df_train_gender['CleanText'] = df_train_gender['Transf_Text'].apply(remove_stopwords)
    df_test_gender['CleanText'] = df_test_gender['Transf_Text'].apply(remove_stopwords)

    df_train_gender["GenderClass"] = df_train_gender["ActualGender"].apply(genderGroups)
    df_test_gender["GenderClass"] = df_test_gender["ActualGender"].apply(genderGroups)

    train_texts_gender = getstringtext(df_train_gender)
    test_texts_gender = getstringtext(df_test_gender)

    tfvec = TfidfVectorizer(max_features = 3000)
    tfvec.fit(train_texts_gender)
    tdf_train = tfvec.transform(train_texts_gender).toarray().tolist()
    tdf_test = tfvec.transform(test_texts_gender).toarray().tolist()

    gender_train = df_train_gender["GenderClass"].to_list()
    gender_test = df_test_gender["GenderClass"].to_list()

    logisticRegr_gender = LogisticRegression(penalty="l2", C=1000)
    logisticRegr_gender.fit(tdf_train, gender_train)

    pred_gender_test=logisticRegr_gender.predict(tdf_test)

    #genero
    df_test_gender["PredictGender"] = pred_gender_test

    # separa as partições de teste com base no predito

    df_train_fem = df_train_gender[df_train_gender["GenderClass"]==1]
    df_test_fem = df_test_gender[df_test_gender["PredictGender"]==1]

    train_texts_fem = getstringtext(df_train_fem)
    test_texts_fem = getstringtext(df_test_fem)
    tfvec_fem = TfidfVectorizer(max_features = 3000)
    tfvec_fem.fit(train_texts_fem)
    tdf_train_fem = tfvec_fem.transform(train_texts_fem).toarray().tolist()
    tdf_test_fem = tfvec_fem.transform(test_texts_fem).toarray().tolist()
    age_train_fem = df_train_fem["AgeClass"].to_list()
    age_test_fem = df_test_fem["AgeClass"].to_list()
    logisticRegr_fem = LogisticRegression(penalty="l2", C=1000)
    logisticRegr_fem.fit(tdf_train_fem, age_train_fem)
    pred_age_fem=logisticRegr_fem.predict(tdf_test_fem)
    ages.append(metrics.f1_score(age_test_fem, pred_age_fem, average='macro'))
    print(metrics.f1_score(age_test_fem, pred_age_fem, average='macro'))

print(sum(ages)/len(ages))

c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7119522945294271


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7036785512167515


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

...

c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5208647561588737


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

0.5555555555555556
0.5969634230503796
0.5801392419899586


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [51]:
ages = []
ages_fem = []
ages_masc = []
ages_sep = []

ages_macro = []
ages_fem_macro = []
ages_masc_macro = []
ages_sep_macro = []

for i in range(10):
    df_train_gender, df_test_gender = createdfs(i+1)

    df_train_gender['CleanText'] = df_train_gender['Transf_Text'].apply(remove_stopwords)
    df_test_gender['CleanText'] = df_test_gender['Transf_Text'].apply(remove_stopwords)

    df_train_gender["GenderClass"] = df_train_gender["ActualGender"].apply(genderGroups)
    df_test_gender["GenderClass"] = df_test_gender["ActualGender"].apply(genderGroups)

    train_texts_gender = getstringtext(df_train_gender)
    test_texts_gender = getstringtext(df_test_gender)

    tfvec = TfidfVectorizer(max_features = 3000)
    tfvec.fit(train_texts_gender)
    tdf_train = tfvec.transform(train_texts_gender).toarray().tolist()
    tdf_test = tfvec.transform(test_texts_gender).toarray().tolist()

    gender_train = df_train_gender["GenderClass"].to_list()
    gender_test = df_test_gender["GenderClass"].to_list()

    logisticRegr_gender = LogisticRegression(penalty=gender_param["k_"+str(i+1)]["params"]["penalty"], C=gender_param["k_"+str(i+1)]["params"]["C"])
    logisticRegr_gender.fit(tdf_train, gender_train)

    pred_gender_test=logisticRegr_gender.predict(tdf_test)

    #genero
    df_test_gender["PredictGender"] = pred_gender_test

    # separa as partições de teste com base no predito

    df_train_masc = df_train_gender[df_train_gender["GenderClass"]==0]
    df_test_masc = df_test_gender[df_test_gender["PredictGender"]==0]

    train_texts_masc = getstringtext(df_train_masc)
    test_texts_masc = getstringtext(df_test_masc)
    tfvec_masc = TfidfVectorizer(max_features = 3000)
    tfvec_masc.fit(train_texts_masc)
    tdf_train_masc = tfvec_masc.transform(train_texts_masc).toarray().tolist()
    tdf_test_masc = tfvec_masc.transform(test_texts_masc).toarray().tolist()
    age_train_masc = df_train_masc["AgeClass"].to_list()
    age_test_masc = df_test_masc["AgeClass"].to_list()
    logisticRegr_masc = LogisticRegression(penalty=age_masc_param["k_"+str(i+1)]["params"]["penalty"], C=age_masc_param["k_"+str(i+1)]["params"]["C"])
    logisticRegr_masc.fit(tdf_train_masc, age_train_masc)
    pred_age_masc=logisticRegr_masc.predict(tdf_test_masc)
    ages_masc_macro.append(metrics.f1_score(age_test_masc, pred_age_masc, average='macro'))
    print(metrics.f1_score(age_test_masc, pred_age_masc, average='macro'))

print(sum(ages_masc_macro)/len(ages_masc_macro))

c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5555555555555555


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


0.6809523809523811


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5493547846489023


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.36996336996337
0.3205128205128205
0.5143461941109


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [48]:
ages = []
ages_fem = []
ages_masc = []
ages_sep = []

ages_macro = []
ages_fem_macro = []
ages_masc_macro = []
ages_sep_macro = []

for i in range(10):
    df_train_gender, df_test_gender = createdfs(i+1)

    df_train_gender['CleanText'] = df_train_gender['Transf_Text'].apply(remove_stopwords)
    df_test_gender['CleanText'] = df_test_gender['Transf_Text'].apply(remove_stopwords)

    df_train_gender["GenderClass"] = df_train_gender["ActualGender"].apply(genderGroups)
    df_test_gender["GenderClass"] = df_test_gender["ActualGender"].apply(genderGroups)

    train_texts_gender = getstringtext(df_train_gender)
    test_texts_gender = getstringtext(df_test_gender)

    tfvec = TfidfVectorizer(max_features = 3000)
    tfvec.fit(train_texts_gender)
    tdf_train = tfvec.transform(train_texts_gender).toarray().tolist()
    tdf_test = tfvec.transform(test_texts_gender).toarray().tolist()

    gender_train = df_train_gender["GenderClass"].to_list()
    gender_test = df_test_gender["GenderClass"].to_list()

    logisticRegr_gender = LogisticRegression(penalty="l2", C=1000)
    logisticRegr_gender.fit(tdf_train, gender_train)

    pred_gender_test=logisticRegr_gender.predict(tdf_test)

    #genero
    df_test_gender["PredictGender"] = pred_gender_test

    # separa as partições de teste com base no predito

    df_train_masc = df_train_gender[df_train_gender["GenderClass"]==0]
    df_test_masc = df_test_gender[df_test_gender["PredictGender"]==0]

    train_texts_masc = getstringtext(df_train_masc)
    test_texts_masc = getstringtext(df_test_masc)
    tfvec_masc = TfidfVectorizer(max_features = 3000)
    tfvec_masc.fit(train_texts_masc)
    tdf_train_masc = tfvec_masc.transform(train_texts_masc).toarray().tolist()
    tdf_test_masc = tfvec_masc.transform(test_texts_masc).toarray().tolist()
    age_train_masc = df_train_masc["AgeClass"].to_list()
    age_test_masc = df_test_masc["AgeClass"].to_list()
    logisticRegr_masc = LogisticRegression(penalty="l2", C=1000)
    logisticRegr_masc.fit(tdf_train_masc, age_train_masc)
    pred_age_masc=logisticRegr_masc.predict(tdf_test_masc)
    ages_masc_macro.append(metrics.f1_score(age_test_masc, pred_age_masc, average='macro'))
    print(metrics.f1_score(age_test_masc, pred_age_masc, average='macro'))

print(sum(ages_masc_macro)/len(ages_masc_macro))

c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5555555555555555


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.4611111111111112


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

...

c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5493547846489023


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

0.36996336996337
0.3205128205128205
0.4940126387773446


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
